In [8]:
# import os
# import openai
# import csv
# from dotenv import load_dotenv

# # Load environment variables
# load_dotenv()

# # Set up OpenAI API
# openai.api_type = "azure"
# openai.api_base = os.getenv("ENDPOINT")
# openai.api_version = "2024-02-15-preview"
# openai.api_key = os.getenv("OPENAI_API_KEY")

# # Create a client
# client = openai.AzureOpenAI(
#     api_key=openai.api_key,
#     api_version=openai.api_version,
#     azure_endpoint=openai.api_base,
# )


# # Function to read file content
# def read_file(file_path):
#     with open(file_path, "r") as file:
#         return file.read()


# # Read input files
# soa_mapping = read_file("outputs/output_activity_to_forms_v2.txt")
# visit_schedule = read_file("outputs/output_study_timeline.txt")
# visit_extractions = read_file("outputs/output_visit_extraction_v1.txt")
# cdash_info = read_file("CDASH-2-3-IG.csv")

# # Construct the prompt
# prompt = f"""
# You are a clinical data management expert tasked with creating a comprehensive Data Management Plan for a clinical trial. You have been provided with four key inputs:

# 1. A table mapping activities from the Schedule of Activities (SOA) to appropriate eCRF forms:
# {soa_mapping}

# 2. A visit schedule extracted from the SOA file:
# {visit_schedule}

# 3. A table of visit extractions including visit names, descriptions, labels, windows, and types:
# {visit_extractions}

# 4. Clinical Data Acquisition Standards Harmonization (CDASH) information for various domains:
# {cdash_info}

# Using these inputs, create a Data Management Plan table with the following columns:
# 1. Check ID (use format DRP-XXXXX)
# 2. Data Page / Data Grouping
# 3. Data Source (typically EDC or Vendor)
# 4. Critical Data Yes/No
# 5. Data Review Type
# 6. Data Review Role
# 7. Data Review Frequency
# 8. Applicable Visits
# 9. Check Description
# 10. Query Text
# 11. Review Comments (leave blank for now)

# For each row in the table:
# - Choose a relevant activity from the SOA mapping.
# - Determine if it's critical data based on its nature and importance in the study.
# - Assign an appropriate data review type (e.g., Eligibility, Missing Data, Data Quality, Protocol Compliance, Safety).
# - Assign a suitable review role (e.g., Data Manager, Medical Monitor, Clinical Monitor) based on the activity.
# - Set a review frequency appropriate for the data type and criticality.
# - Specify applicable visits based on the visit schedule and extractions.
# - Write a clear check description related to the data point.
# - Create a specific query text that would be used if the check fails.

# Ensure that your Data Management Plan covers a range of important study activities, includes both critical and non-critical data, and addresses various aspects of data review and quality control. The plan should be tailored to the specific study based on the provided inputs.

# Please generate a Data Management Plan table with at least 10 rows, formatted as a CSV for easy reading and further processing.
# """

Split the task into multiple API calls, since the data is too big to be processed.

In [15]:
# import os
# import openai
# import csv
# from dotenv import load_dotenv
# import re

# # Load environment variables
# load_dotenv()

# # Set up OpenAI API
# openai.api_type = "azure"
# openai.api_base = os.getenv("ENDPOINT")
# openai.api_version = "2024-02-15-preview"
# openai.api_key = os.getenv("OPENAI_API_KEY")

# # Create a client
# client = openai.AzureOpenAI(
#     api_key=openai.api_key,
#     api_version=openai.api_version,
#     azure_endpoint=openai.api_base,
# )


# def clean_output(text):
#     # Remove any lines that start with "Certainly!" or similar phrases
#     lines = text.split("\n")
#     cleaned_lines = [
#         line
#         for line in lines
#         if not re.match(r"^(Certainly!|Here are|Sure,)", line.strip())
#     ]

#     # Remove any content enclosed in triple backticks
#     cleaned_text = re.sub(r"```[\s\S]*?```", "", "\n".join(cleaned_lines))

#     # Remove any remaining empty lines
#     cleaned_text = "\n".join(line for line in cleaned_text.split("\n") if line.strip())

#     return cleaned_text


# # Function to read file content
# def read_file(file_path):
#     with open(file_path, "r") as file:
#         return file.read()


# # Read input files
# soa_mapping = read_file("outputs/output_activity_to_forms_v2.txt")
# visit_schedule = read_file("outputs/output_study_timeline.txt")
# visit_extractions = read_file("outputs/output_visit_extraction_v1.txt")
# cdash_info = read_file("CDASH-2-3-IG.csv")


# # Function to call OpenAI API
# def call_openai_api(prompt):
#     response = client.chat.completions.create(
#         model="gpt-4o",  # Make sure this is the correct model name for your Azure deployment
#         messages=[
#             {"role": "system", "content": "You are a clinical data management expert."},
#             {"role": "user", "content": prompt},
#         ],
#         temperature=0.7,
#         max_tokens=2000,
#     )
#     return response.choices[0].message.content


# # Function to chunk large text
# def chunk_text(text, chunk_size=500000):
#     return [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]



In [16]:
# def generate_data_management_plan(
#     soa_mapping, visit_schedule, visit_extractions, cdash_info
# ):
#     header = "Check ID\tData Page / Data Grouping\tData Source\tCritical Data Yes/No\tData Review Type\tData Review Role\tData Review Frequency\tApplicable Visits\tCheck Description\tQuery Text\tReview Comments"

#     initial_prompt = f"""Create a Data Management Plan table for a clinical trial. Use the following header:

# {header}

# Generate 5 rows for the Data Management Plan. Each row should follow this format and be separated by tabs. Ensure that:
# 1. Check IDs follow the format DRP-XXXXX (where XXXXX is a 5-digit number).
# 2. Data Source is typically EDC or Vendor.
# 3. Critical Data is either Yes or No.
# 4. Data Review Role can be Data Manager, Clinical Monitor, Medical Monitor, etc.
# 5. Data Review Frequency can be Daily, Weekly, Bi-weekly, Monthly, etc.
# 6. Applicable Visits should be specific (e.g., SCR 1, Cycle 1 Day 1, EOT, etc.).
# 7. The Review Comments column should be left empty.

# Ensure diversity in data groupings, review types, and visit applicability."""

#     table_rows = call_openai_api(initial_prompt)
#     full_table = header + "\n" + table_rows

#     # Process SOA mapping
#     soa_prompt = f"""Based on this SOA mapping:

# {soa_mapping}

# Add 5 more rows to the Data Management Plan table. Follow the same format as before, ensuring diversity in data groupings, review types, and visit applicability. Provide only the new rows, separated by tabs."""

#     soa_rows = call_openai_api(soa_prompt)
#     full_table += "\n" + soa_rows

#     # Process visit schedule
#     visit_prompt = f"""Based on this visit schedule:

# {visit_schedule}

# Add 5 more rows to the Data Management Plan table, focusing on visit-specific checks. Follow the same format as before. Provide only the new rows, separated by tabs."""

#     visit_rows = call_openai_api(visit_prompt)
#     full_table += "\n" + visit_rows

#     # Process visit extractions
#     extraction_prompt = f"""Based on these visit extractions:

# {visit_extractions}

# Add 5 more rows to the Data Management Plan table, focusing on data quality and completeness checks. Follow the same format as before. Provide only the new rows, separated by tabs."""

#     extraction_rows = call_openai_api(extraction_prompt)
#     full_table += "\n" + extraction_rows

#     # Process CDASH info
#     cdash_prompt = f"""Based on this CDASH information:

# {cdash_info[:1000]}  # Using only the first 1000 characters to avoid token limit

# Add 5 more rows to the Data Management Plan table, focusing on CDASH-specific data checks. Follow the same format as before. Provide only the new rows, separated by tabs."""

#     cdash_rows = call_openai_api(cdash_prompt)
#     full_table += "\n" + cdash_rows

#     return full_table


# # Generate the Data Management Plan
# generated_table = generate_data_management_plan(
#     soa_mapping, visit_schedule, visit_extractions, cdash_info
# )

# # Save the generated table as a CSV file
# with open("data_management_plan.csv", "w", newline="", encoding="utf-8") as file:
#     file.write(generated_table)

# print(
#     "Data Management Plan has been generated and saved as 'data_management_plan.csv'."
# )

Data Management Plan has been generated and saved as 'data_management_plan.csv'.


Filter by domain_name and field_core(hr and r/c)

In [21]:
# import os
# import openai
# from dotenv import load_dotenv
# import re
# import csv
# from io import StringIO

# # Load environment variables
# load_dotenv()

# # Set up OpenAI API
# openai.api_type = "azure"
# openai.api_base = os.getenv("ENDPOINT")
# openai.api_version = "2024-02-15-preview"
# openai.api_key = os.getenv("OPENAI_API_KEY")

# # Create a client
# client = openai.AzureOpenAI(
#     api_key=openai.api_key,
#     api_version=openai.api_version,
#     azure_endpoint=openai.api_base
# )

# # Function to read file content
# def read_file(file_path):
#     with open(file_path, "r") as file:
#         return file.read()

# # Read input files
# soa_mapping = read_file("outputs/output_activity_to_forms_v2.txt")
# visit_schedule = read_file("outputs/output_study_timeline.txt")
# visit_extractions = read_file("outputs/output_visit_extraction_v1.txt")
# cdash_info = read_file("CDASH-2-3-IG.csv")

# def call_openai_api(prompt):
#     response = client.chat.completions.create(
#         model="gpt-4o",  # Make sure this is the correct model name for your Azure deployment
#         messages=[
#             {"role": "system", "content": "You are a clinical data management expert. Provide only the requested table rows without any additional text or formatting."},
#             {"role": "user", "content": prompt},
#         ],
#         temperature=0.7,
#         max_tokens=2000,
#     )
#     return response.choices[0].message.content

# def clean_output(text):
#     # Remove any lines that don't look like our expected data
#     lines = text.split('\n')
#     cleaned_lines = [line for line in lines if re.match(r'^DRP-\d{5}\t', line)]
#     return '\n'.join(cleaned_lines)

# def filter_and_chunk_cdash_by_domain(cdash_info):
#     # Parse the CSV data
#     csv_data = csv.reader(StringIO(cdash_info))
#     header = next(csv_data)
    
#     # Find the indices of the relevant columns
#     domain_index = header.index('domain_name')
#     core_index = header.index('field_core')
    
#     # Group rows by domain_name, filtering for HR and R/C fields
#     domains = {}
#     for row in csv_data:
#         if row[core_index] in ['HR', 'R/C']:
#             domain = row[domain_index]
#             if domain not in domains:
#                 domains[domain] = [header]
#             domains[domain].append(row)
    
#     # Convert each domain group back to CSV string
#     return {domain: '\n'.join([','.join(row) for row in rows]) for domain, rows in domains.items()}

# def generate_data_management_plan(soa_mapping, visit_schedule, visit_extractions, cdash_info):
#     header = "Check ID\tData Page / Data Grouping\tData Source\tCritical Data Yes/No\tData Review Type\tData Review Role\tData Review Frequency\tApplicable Visits\tCheck Description\tQuery Text\tReview Comments"
    
#     full_table = header + "\n"

#     # Process SOA mapping
#     soa_prompt = f"""Based on this SOA mapping:

# {soa_mapping}

# Create rows for a Data Management Plan table. Each row should follow this format and be separated by tabs:

# {header}

# Ensure that:
# 1. Check IDs follow the format DRP-XXXXX (where XXXXX is a 5-digit number).
# 2. Data Source is typically EDC or Vendor.
# 3. Critical Data is either Yes or No.
# 4. Data Review Role can be Data Manager, Clinical Monitor, Medical Monitor, etc.
# 5. Data Review Frequency can be Daily, Weekly, Bi-weekly, Monthly, etc.
# 6. Applicable Visits should be specific (e.g., SCR 1, Cycle 1 Day 1, EOT, etc.).
# 7. The Review Comments column should be left empty.

# Provide only the data rows, without the header or any additional text."""

#     soa_rows = call_openai_api(soa_prompt)
#     full_table += clean_output(soa_rows) + "\n"

#     # Process visit schedule
#     visit_prompt = f"""Based on this visit schedule:

# {visit_schedule}

# Add more rows to the Data Management Plan table, focusing on visit-specific checks. Follow the same format as before. Provide only the new rows, without any additional text."""

#     visit_rows = call_openai_api(visit_prompt)
#     full_table += clean_output(visit_rows) + "\n"

#     # Process visit extractions
#     extraction_prompt = f"""Based on these visit extractions:

# {visit_extractions}

# Add more rows to the Data Management Plan table, focusing on data quality and completeness checks. Follow the same format as before. Provide only the new rows, without any additional text."""

#     extraction_rows = call_openai_api(extraction_prompt)
#     full_table += clean_output(extraction_rows) + "\n"

#     # Process CDASH info by domain
#     cdash_chunks = filter_and_chunk_cdash_by_domain(cdash_info)
#     for domain, chunk in cdash_chunks.items():
#         cdash_prompt = f"""Based on the CDASH information for the {domain} domain:

# {chunk}

# Add rows to the Data Management Plan table, focusing on CDASH-specific data checks for this domain. Consider only the Highly Recommended (HR) and Required/Conditional (R/C) fields. Follow the same format as before. Provide only the new rows, without any additional text."""

#         cdash_rows = call_openai_api(cdash_prompt)
#         full_table += clean_output(cdash_rows) + "\n"

#     return full_table

# # Generate the Data Management Plan
# generated_table = generate_data_management_plan(soa_mapping, visit_schedule, visit_extractions, cdash_info)

# # Save the generated table as a CSV file
# with open("data_management_plan.csv", "w", newline="", encoding='utf-8') as file:
#     file.write(generated_table)

# print("Data Management Plan has been generated and saved as 'data_management_plan.csv'.")

Data Management Plan has been generated and saved as 'data_management_plan.csv'.


Processing time is too long, add those method:
- Parallel processing: use `concurrent.futures.ThreadPoolExecutor` to make API calls concurrently.
- Caching: add `@lru_cache` to the `call_openai_api` function to cache responses and avoid redundant API calls.
- Batching: batch CDASH domains together (3 domains per batch) to reduce the number of API calls.

In [22]:
import os
import openai
from dotenv import load_dotenv
import re
import csv
from io import StringIO
import concurrent.futures
import json
from functools import lru_cache

# Load environment variables
load_dotenv()

# Set up OpenAI API
openai.api_type = "azure"
openai.api_base = os.getenv("ENDPOINT")
openai.api_version = "2024-02-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Create a client
client = openai.AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint=openai.api_base,
)


# Function to read file content
def read_file(file_path):
    with open(file_path, "r") as file:
        return file.read()


# Read input files
soa_mapping = read_file("outputs/output_activity_to_forms_v2.txt")
visit_schedule = read_file("outputs/output_study_timeline.txt")
visit_extractions = read_file("outputs/output_visit_extraction_v1.txt")
cdash_info = read_file("CDASH-2-3-IG.csv")


@lru_cache(maxsize=None)
def call_openai_api(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[
            {
                "role": "system",
                "content": "You are a clinical data management expert. Provide only the requested table rows without any additional text or formatting.",
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
        max_tokens=4000,
    )
    return response.choices[0].message.content


def clean_output(text):
    lines = text.split("\n")
    cleaned_lines = [line for line in lines if re.match(r"^DRP-\d{5}\t", line)]
    return "\n".join(cleaned_lines)


def filter_and_chunk_cdash_by_domain(cdash_info):
    csv_data = csv.reader(StringIO(cdash_info))
    header = next(csv_data)

    domain_index = header.index("domain_name")
    core_index = header.index("field_core")
    name_index = header.index("field_name")
    label_index = header.index("field_label")

    domains = {}
    for row in csv_data:
        if row[core_index] in ["HR", "R/C"]:
            domain = row[domain_index]
            if domain not in domains:
                domains[domain] = []
            domains[domain].append(f"{row[name_index]}: {row[label_index]}")

    return domains


def process_chunk(chunk_type, chunk_data):
    header = "Check ID\tData Page / Data Grouping\tData Source\tCritical Data Yes/No\tData Review Type\tData Review Role\tData Review Frequency\tApplicable Visits\tCheck Description\tQuery Text\tReview Comments"

    if chunk_type == "soa":
        prompt = f"""Based on this SOA mapping:

{chunk_data}

Create rows for a Data Management Plan table. Each row should follow this format and be separated by tabs:

{header}

Ensure that:
1. Check IDs follow the format DRP-XXXXX (where XXXXX is a 5-digit number).
2. Data Source is typically EDC or Vendor.
3. Critical Data is either Yes or No.
4. Data Review Role can be Data Manager, Clinical Monitor, Medical Monitor, etc.
5. Data Review Frequency can be Daily, Weekly, Bi-weekly, Monthly, etc.
6. Applicable Visits should be specific (e.g., SCR 1, Cycle 1 Day 1, EOT, etc.).
7. The Review Comments column should be left empty.

Provide only the data rows, without the header or any additional text."""
    elif chunk_type == "visit":
        prompt = f"""Based on this visit information:

{chunk_data}

Add more rows to the Data Management Plan table, focusing on visit-specific checks. Follow the same format as before. Provide only the new rows, without any additional text."""
    elif chunk_type == "cdash":
        prompt = f"""Based on the CDASH information for the following domains:

{json.dumps(chunk_data, indent=2)}

Add rows to the Data Management Plan table, focusing on CDASH-specific data checks for these domains. Consider only the Highly Recommended (HR) and Required/Conditional (R/C) fields provided. Follow the same format as before. Provide only the new rows, without any additional text."""

    rows = call_openai_api(prompt)
    return clean_output(rows)


def generate_data_management_plan(
    soa_mapping, visit_schedule, visit_extractions, cdash_info
):
    header = "Check ID\tData Page / Data Grouping\tData Source\tCritical Data Yes/No\tData Review Type\tData Review Role\tData Review Frequency\tApplicable Visits\tCheck Description\tQuery Text\tReview Comments"

    cdash_chunks = filter_and_chunk_cdash_by_domain(cdash_info)

    # Batch CDASH domains
    batched_cdash = {}
    batch_size = 3
    for i in range(0, len(cdash_chunks), batch_size):
        batch = list(cdash_chunks.items())[i : i + batch_size]
        batched_cdash[f"batch_{i//batch_size}"] = dict(batch)

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        futures.append(executor.submit(process_chunk, "soa", soa_mapping))
        futures.append(executor.submit(process_chunk, "visit", visit_schedule))
        futures.append(executor.submit(process_chunk, "visit", visit_extractions))
        for batch in batched_cdash.values():
            futures.append(executor.submit(process_chunk, "cdash", batch))

        results = [
            future.result() for future in concurrent.futures.as_completed(futures)
        ]

    full_table = header + "\n" + "\n".join(results)
    return full_table


# Generate the Data Management Plan
generated_table = generate_data_management_plan(
    soa_mapping, visit_schedule, visit_extractions, cdash_info
)

# Save the generated table as a CSV file
with open("data_management_plan.csv", "w", newline="", encoding="utf-8") as file:
    file.write(generated_table)

print(
    "Data Management Plan has been generated and saved as 'data_management_plan.csv'."
)

Data Management Plan has been generated and saved as 'data_management_plan.csv'.
